In [54]:
import pandas as pd
import numpy as np
import tensorflow

In [61]:
import pandas as pd
import numpy as np
import tensorflow
import matplotlib.pyplot as plt
import seaborn as sns

In [60]:
Data = pd.read_csv("/content/drive/MyDrive/Data/spam.csv", encoding="latin-1")
Data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [16]:
Data.describe().T

,count,unique,top,freq
v1,5572,2,ham,4825
v2,5572,5169,"Sorry, I'll call later",30


In [17]:
Data.isnull().sum()

v1    0
v2    0
dtype: int64

In [58]:
Data.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)

In [59]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


# CREATE THE MODEL

In [32]:
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
X = Data.v2
Y = Data.v1
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
from keras.layers import Dense , LSTM , Embedding , Dropout , Activation , Flatten
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.2)

In [35]:
from keras.models import Sequential
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop

In [38]:
max_word = 1000
max_len = 250
token = Tokenizer(num_words = max_word)
token.fit_on_texts(X_train)
sequences = token.texts_to_sequences(X_train)
seq_matrix = sequence.pad_sequences(sequences , maxlen = max_len)

# ADDING LAYERS

* LSTM

* DENSE-(Hiddden Layers)

* OUTPUT 










In [40]:
model = Sequential()
model.add(Embedding(max_word , 32 , input_length = max_len))
model.add(LSTM(64))
model.add(Flatten())

In [41]:
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(120, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# COMPILE THE MODEL

In [42]:
model.compile(loss = 'binary_crossentropy' , optimizer = 'RMSprop' , metrics = 'accuracy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 32)           32000     
                                                                 
 lstm_1 (LSTM)               (None, 64)                24832     
                                                                 
 flatten_1 (Flatten)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 250)               16250     
                                                                 
 dropout (Dropout)           (None, 250)               0         
                                                                 
 dense_1 (Dense)             (None, 120)               30120     
                                                                 
 dense_2 (Dense)             (None, 1)                

# FIT THE MODEL

In [43]:
model.fit(seq_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
28/28 [==============================] - 10s 254ms/step - loss: 0.3621 - accuracy: 0.8519 - val_loss: 0.1717 - val_accuracy: 0.9283
Epoch 2/10
28/28 [==============================] - 7s 269ms/step - loss: 0.0945 - accuracy: 0.9748 - val_loss: 0.0473 - val_accuracy: 0.9854


# SAVE THE MODEL

In [44]:
model.save("spam.h5")

# TEST THE MODEL

In [49]:
from tensorflow.keras.models import load_model
model1 = load_model("spam.h5")

In [47]:
test_seq = token.texts_to_sequences(X_test)
test_seq_matrix = sequence.pad_sequences(test_seq,maxlen=max_len)

In [48]:
scores = model.evaluate(test_seq_matrix, Y_test, verbose=0)
scores

[0.0654357373714447, 0.9793722033500671]